In [ ]:
!pip install numpy==1.26.4
!pip install sentence-transformers==2.2.2
!pip install transformers==4.40.1 huggingface_hub==0.22.2
!pip install TTS==0.22.0
!pip install -r https://raw.githubusercontent.com/coqui-ai/TTS/dev/requirements.txt
!pip install faiss-cpu==1.7.4
!pip install -q git+https://github.com/openai/whisper.git
!pip install -q fastapi==0.100.0 uvicorn==0.22.0 python-multipart
!pip install telethon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 23.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
thinc 8.3.6 requires numpy<3.0.0,>=2.0.0, but you have numpy 1.26.4 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 4.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 124.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 63.7 MB/

In [ ]:
# إزالة النسخة القديمة إن وُجدت
!rm -f ngrok ngrok.zip ngrok-v3.tgz

# تحميل أحدث نسخة من ngrok v3
!wget -q -O ngrok-v3.tgz https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-v3-stable-linux-amd64.tgz

# فك الضغط
!tar -xvzf ngrok-v3.tgz


ngrok


In [ ]:
!./ngrok version


ngrok version 3.23.0


In [ ]:
# استبدل النص التالي بالتوكن الخاص بك من لوحة ngrok
!./ngrok config add-authtoken 2jWVfjzQtDSJm3YlDOQUoDV6Lz3_53ejySSv1WMG1jVWm14vz


Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [ ]:
pip install gTTS


In [ ]:
from gtts import gTTS

In [ ]:
def convert_text_to_audio(text: str) -> str:
    # استخدام gTTS لتوليد الصوت
    tts = gTTS(text=text, lang='en')  # إضافة اللغة العربية 'ar'
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as f:
        audio_path = f.name
        tts.save(audio_path)  # حفظ الصوت في الملف
    return audio_path

In [ ]:
# اخر حاجة
from fastapi import FastAPI, UploadFile, File, Form, HTTPException
from fastapi.responses import FileResponse
from pydantic import BaseModel, Field
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import requests
import uvicorn
import os
import whisper
import tempfile
from TTS.api import TTS as CoquiTTS
import nest_asyncio
import asyncio
import subprocess
import time
import json
from pathlib import Path
import httpx # استيراد المكتبة الجديدة

import base64
import io

from telethon.errors import SessionPasswordNeededError
from telethon.sessions import StringSession
from telethon.sync import TelegramClient

nest_asyncio.apply()

app = FastAPI()


# Ngrok setup (optional, comment out if not needed or run ngrok separately)
# Ensure ngrok executable is in the same directory or in PATH
try:
    ngrok_process = subprocess.Popen(["./ngrok", "http", "8000"])
    time.sleep(3) # Give ngrok some time to start
    # Try fetching the public URL from ngrok's API
    try:
        r = requests.get("http://localhost:4040/api/tunnels")
        r.raise_for_status() # Raise an exception for HTTP errors
        public_url = r.json()["tunnels"][0]["public_url"]
        print(f"ngrok public URL: {public_url}")
    except requests.exceptions.RequestException as e:
        print(f"Could not connect to ngrok API: {e}. Ngrok might still be running.")
        print("Check ngrok status manually if needed.")
    except (KeyError, IndexError) as e:
        print(f"Could not parse ngrok API response: {e}")
        print("ngrok JSON response:", r.text)
except FileNotFoundError:
    print("ngrok executable not found. Skipping ngrok startup.")
except Exception as e:
    print(f"An error occurred during ngrok setup: {e}")


# # Global models (loaded once)
# try:
#     whisper_model = whisper.load_model("large-v3")
#     embedding_model = SentenceTransformer('all-MiniLM-L6-v2') # Renamed for clarity
#     # Initialize one TTS model. The second one in original code would overwrite the first.
#     # Using fast_pitch as it was the last one assigned.
#     tts_model = CoquiTTS(model_name="tts_models/en/ljspeech/fast_pitch")
#     # tts_model = CoquiTTS(model_name="tts_models/en/ljspeech/tacotron2-DDC")
#     # tts_model = CoquiTTS(model_name="tts_models/en/vctk/vits", progress_bar=True)
# except Exception as e:
#     print(f"Error loading one or more models: {e}")
#     print("Please ensure model files are correctly downloaded and accessible.")
#     # Potentially exit or disable features if models can't load
#     # For now, we'll let it continue and it will fail later if these models are used.


# GROQ_API_KEY = os.environ.get("GROQ_API_KEY", "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") # Use environment variable or default
# if GROQ_API_KEY == "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" and not os.environ.get("GROQ_API_KEY"):
#     print("WARNING: Using a default GROQ_API_KEY. Please set your own key via an environment variable.")

GROQ_API_KEY = "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"

BASE_CHARACTER_DATA_PATH = Path("data_storage/characters")
BASE_CHARACTER_DATA_PATH.mkdir(parents=True, exist_ok=True) # Ensure base directory exists

# --- إعدادات تيليجرام (الخاصة بك كمطور) ---
API_ID = 24838737  # استبدل بـ api_id الخاص بك
API_HASH = 'dd8fe04fbfb8db4e7f0b8e379adc4971'  # استبدل بـ api_hash الخاص بك
pending_clients = {} # لحفظ جلسات تسجيل الدخول المؤقتة






# --- Pydantic Models ---
class CharacterProfileData(BaseModel):
    characterId: str
    characterName: str
    bio: str
    answers: dict

class WhatsAppMessage(BaseModel):
    id: str
    from_user: str = Field(..., alias='from')
    to: str
    timestamp: int
    body: str

class WhatsAppPayload(BaseModel):
    characterName: str
    characterId: str
    messages: list[WhatsAppMessage]

# --- Pydantic Models ---
class PhoneRequest(BaseModel):
    phone_number: str

class LoginRequest(BaseModel):
    phone_number: str
    code: str
    password: str | None = None

class ProcessChatRequest(BaseModel):
    chatId: int

# --- AI Helper Functions ---
async def call_groq_api(system_prompt: str, user_prompt: str) -> str:
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "llama-3.3-70b-versatile",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
        ],
        "temperature": 0.3,
    }
    try:
        async with httpx.AsyncClient() as client:
            response = await client.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=payload, timeout=45.0)
            response.raise_for_status()
            return response.json()["choices"][0]["message"]["content"]
    except httpx.RequestError as e:
        print(f"Groq API request error: {e}")
        raise HTTPException(status_code=500, detail="Failed to connect to Groq API.")
    except (KeyError, IndexError) as e:
        print(f"Error parsing Groq API response: {e}")
        raise HTTPException(status_code=500, detail="Invalid response from Groq API.")

async def summarize_chat_for_bio(text: str, character_name: str) -> str:
    system_prompt = "You are a skilled biographer. Your task is to analyze the provided text, which contains messages from a person, and write a concise, descriptive, third-person biography about them. Focus on their personality, way of speaking, and any personal details mentioned."
    user_prompt = f"Based on the following messages from a person named '{character_name}', write their biography:\n\n---\n{text}\n---"
    return await call_groq_api(system_prompt, user_prompt)

async def extract_qna_from_chat(text: str) -> dict:
    system_prompt = "You are a data extraction specialist. Analyze the provided text and extract key personal details (like name, age, job, location, interests, etc.) into a valid JSON object. If a detail is not mentioned, do not include it. The keys should be in English. Respond ONLY with the raw JSON object and nothing else."
    user_prompt = f"Extract personal details from this text into a JSON object:\n\n---\n{text}\n---"
    json_string = await call_groq_api(system_prompt, user_prompt)
    try:
        clean_json_string = json_string.strip().replace("```json", "").replace("```", "")
        return json.loads(clean_json_string)
    except json.JSONDecodeError:
        print(f"Warning: Failed to decode JSON from Groq response: {json_string}")
        return {}

# --- Helper Functions ---
def convert_audio_to_text(audio_bytes: bytes) -> str:
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as temp_audio: # Common audio format
        temp_audio.write(audio_bytes)
        temp_audio.flush()
        audio_path = temp_audio.name
    try:
        result = whisper_model.transcribe(audio_path)
        transcribed_text = result['text']
    except Exception as e:
        print(f"Error during audio transcription: {e}")
        transcribed_text = "" # Return empty string or raise error
    finally:
        os.remove(audio_path)
    return transcribed_text

# def convert_text_to_audio(text: str) -> str | None:
#     try:
#         # Create a temporary file that CoquiTTS can write to.
#         # Suffix must be .wav as specified in tts_to_file or supported by the model
#         with tempfile.NamedTemporaryFile(delete=False, suffix=".wav") as f:
#             audio_path = f.name
#         # tts_to_file will create/overwrite this file
#         tts_model.tts_to_file(text=text, file_path=audio_path)
#         return audio_path
#     except Exception as e:
#         print(f"Error during text-to-speech conversion: {e}")
#         return None

# --- API Endpoints ---
# --- NEW ENDPOINT for WhatsApp Preprocessing ---
@app.post("/preprocessing_whats_data")
async def preprocess_whatsapp_data(payload: WhatsAppPayload):
    user_number = "201505307996@c.us"
    target_number = next((msg.from_user for msg in payload.messages if msg.from_user != user_number), None)

    if not target_number:
        raise HTTPException(status_code=400, detail="Could not identify the target user in the chat.")

    target_messages_text_full = "\n".join(
        [msg.body for msg in payload.messages if msg.from_user == target_number and msg.body.strip()]
    )

    if not target_messages_text_full:
        raise HTTPException(status_code=400, detail="No messages found from the target user to process.")

    max_chars = 16000
    target_messages_text_truncated = target_messages_text_full[:max_chars]

    print(f"Processing {len(target_messages_text_truncated)} characters for AI analysis...")

    bio_task = summarize_chat_for_bio(target_messages_text_truncated, payload.characterName)
    answers_task = extract_qna_from_chat(target_messages_text_truncated)

    generated_bio, generated_answers = await asyncio.gather(bio_task, answers_task)

    character_data = CharacterProfileData(
        characterId=payload.characterId,
        characterName=payload.characterName,
        bio=generated_bio,
        answers=generated_answers
    )

    try:
        # استدعاء /store_data لتخزين البيانات في ملفات الباك إند
        await store_data_endpoint(character_data)

        # --- هذا هو الجزء الحاسم والمصحح ---
        # يقوم بإرجاع كائن يحتوي على processedData
        return {
            "status": "success",
            "message": f"Successfully preprocessed and stored data for {payload.characterName}.",
            "processedData": {
                "bio": generated_bio,
                "answers": generated_answers
            }
        }
        # ------------------------------------

    except Exception as e:
        print(f"Error calling store_data_endpoint internally: {e}")
        raise HTTPException(status_code=500, detail=f"Failed to store preprocessed data: {str(e)}")





# --- Endpoints تسجيل الدخول لتيليجرام ---
@app.post("/telegram/send_code")
async def telegram_send_code(req: PhoneRequest):
    client = TelegramClient(StringSession(), API_ID, API_HASH)
    await client.connect()
    try:
        code_hash_data = await client.send_code_request(req.phone_number)
        pending_clients[req.phone_number] = (client, code_hash_data.phone_code_hash)
        return {"status": "code_sent"}
    except Exception as e:
        await client.disconnect()
        raise HTTPException(status_code=500, detail=str(e))

@app.post("/telegram/login")
async def telegram_login(req: LoginRequest):
    if req.phone_number not in pending_clients:
        raise HTTPException(status_code=404, detail="Session not found. Please request a code first.")

    client, phone_code_hash = pending_clients[req.phone_number]
    try:
        await client.sign_in(req.phone_number, req.code, phone_code_hash=phone_code_hash)
    except SessionPasswordNeededError:
        if not req.password:
            return {"status": "password_required"}
        await client.sign_in(password=req.password)
    except Exception as e:
        raise HTTPException(status_code=400, detail=str(e))

    session_string = client.session.save()
    del pending_clients[req.phone_number]
    await client.disconnect()
    return {"status": "login_successful", "sessionString": session_string}

# --- Endpoints التعامل مع الشاتات ---
@app.get("/telegram/chats/{session_string}")
async def telegram_get_chats(session_string: str):
    async with TelegramClient(StringSession(session_string), API_ID, API_HASH) as client:
        chats = []
        dialogs = await client.get_dialogs(limit=70)
        for dialog in dialogs:
            if dialog.is_user and not dialog.entity.bot:
                name = dialog.name
                if hasattr(dialog.entity, 'first_name') and dialog.entity.first_name:
                    name = f"{dialog.entity.first_name} {dialog.entity.last_name or ''}".strip()

                # جلب الصورة كـ bytes وتحويلها إلى Base64 Data URI
                pic_base64 = None
                try:
                    buffer = io.BytesIO()
                    await client.download_profile_photo(dialog.entity, file=buffer)
                    if buffer.getbuffer().nbytes > 0:
                        encoded_pic = base64.b64encode(buffer.getvalue()).decode('utf-8')
                        pic_base64 = f"data:image/jpeg;base64,{encoded_pic}"
                except Exception:
                    pass # تجاهل إذا لم توجد صورة

                chats.append({"id": dialog.id, "name": name, "picUrl": pic_base64})
        return {"status": "success", "chats": chats}

@app.post("/telegram/process_chat/{session_string}")
async def telegram_process_chat(session_string: str, payload: ProcessChatRequest):
    # إنشاء عميل باستخدام الجلسة المحفوظة
    async with TelegramClient(StringSession(session_string), API_ID, API_HASH) as client:
        try:
            # --- السطر الحاسم الذي تمت إضافته ---
            # هذا السطر يقوم بتحميل كل الشاتات في ذاكرة الجلسة الحالية
            # مما يجعل Telethon يتعرف على أي ID نمرره له لاحقًا.
            print("Warming up session entity cache by fetching dialogs...")
            await client.get_dialogs()
            print("Entity cache is ready.")
            # --- نهاية الإضافة ---

            # 1. جلب الرسائل
            chat_id = payload.chatId
            messages = []
            user_entity = await client.get_entity(chat_id)
            me = await client.get_me()

            print(f"Fetching messages from chat with {user_entity.first_name}...")
            async for message in client.iter_messages(chat_id, limit=200):
                if message.text: # نهتم بالرسائل النصية فقط حاليًا
                    from_user = str(me.id) if message.out else str(user_entity.id)
                    to_user = str(user_entity.id) if message.out else str(me.id)
                    messages.append({
                        "id": str(message.id),
                        "from": from_user,
                        "to": to_user,
                        "timestamp": int(message.date.timestamp()),
                        "body": message.text
                    })

            print(f"Fetched {len(messages)} messages. Starting AI processing...")
            # 2. استدعاء الذكاء الاصطناعي (نفس المنطق السابق)
            character_name = f"{user_entity.first_name} {user_entity.last_name or ''}".strip()
            target_messages_text = "\n".join([m['body'] for m in messages if m['from'] != str(me.id)])

            if not target_messages_text:
                raise HTTPException(status_code=400, detail="No messages from target user to process.")

            # ... باقي الكود الخاص بالمعالجة والـ return يبقى كما هو ...
            bio_task = summarize_chat_for_bio(target_messages_text, character_name)
            answers_task = extract_qna_from_chat(target_messages_text)
            generated_bio, generated_answers = await asyncio.gather(bio_task, answers_task)

            # 3. إرجاع بيانات الشخصية الكاملة
            character_id_str = f"tg_{user_entity.id}"
            return {
                "status": "success",
                "characterProfile": {
                    "id": character_id_str,
                    "name": character_name,
                    "bio": generated_bio,
                    "questionnaireAnswers": generated_answers,
                }
            }

        except Exception as e:
            # طباعة الخطأ في سيرفر البايثون لتسهيل التشخيص
            print(f"An error occurred in telegram_process_chat: {e}")
            # إرجاع الخطأ للفرونت إند
            raise HTTPException(status_code=500, detail=str(e))






@app.post("/store_data")
async def store_data_endpoint(data: CharacterProfileData): # Renamed for clarity
    char_id = data.characterId
    char_path = BASE_CHARACTER_DATA_PATH / char_id
    char_path.mkdir(parents=True, exist_ok=True)

    # 1. Save profile.json
    profile_data = {"characterName": data.characterName, "bio": data.bio}
    with open(char_path / "profile.json", "w", encoding="utf-8") as f:
        json.dump(profile_data, f, ensure_ascii=False, indent=4)

    # 2. Save questionnaire.json
    with open(char_path / "questionnaire.json", "w", encoding="utf-8") as f:
        json.dump(data.answers, f, ensure_ascii=False, indent=4)

    # 3. Prepare texts for FAISS
    texts_to_index = []
    texts_to_index.append(f"My name is {data.characterName}.")
    texts_to_index.append(f"My detailed biography is: {data.bio}")
    for q_key, q_ans in data.answers.items():
        texts_to_index.append(f"For the question about '{q_key}', the answer is '{q_ans}'.")

    # 4. Create/Update FAISS index (index.faiss) and texts.json for this character
    if texts_to_index:
        try:
            embeddings = embedding_model.encode(texts_to_index)
            if embeddings.ndim == 1: # Ensure embeddings is 2D
                 embeddings = np.asarray([embeddings])
            if embeddings.shape[0] > 0: # Check if there are any embeddings
                char_faiss_index = faiss.IndexFlatL2(embeddings.shape[1])
                char_faiss_index.add(embeddings)
                faiss.write_index(char_faiss_index, str(char_path / "index.faiss"))
                with open(char_path / "texts.json", "w", encoding="utf-8") as f:
                    json.dump(texts_to_index, f, ensure_ascii=False, indent=4)
                message = f"Data for {data.characterName} stored/updated with FAISS index!"
            else:
                message = f"Data for {data.characterName} stored, but no text was provided to create a FAISS index."

        except Exception as e:
            print(f"Error creating FAISS index for {data.characterName}: {e}")
            return {"status": "error", "message": f"Error creating FAISS index: {e}"}
    else:
        message = f"Data for {data.characterName} stored, but no text content to index."
        # Optionally remove old index files if no new text is provided
        if (char_path / "index.faiss").exists():
            (char_path / "index.faiss").unlink()
        if (char_path / "texts.json").exists():
            (char_path / "texts.json").unlink()


    return {"status": "success", "message": message}

# async def process_chat_or_call(
#     characterId: str,
#     user_message_text: str | None = None,
#     user_audio_file: UploadFile | None = None,
#     chat_history_str: str | None = None,
#     is_call: bool = False
# ):
#     char_path = BASE_CHARACTER_DATA_PATH / characterId
#     if not char_path.exists() or not (char_path / "profile.json").exists():
#         return {"error": "Character not found or profile is missing."}

#     # 1. Load character profile
#     try:
#         with open(char_path / "profile.json", "r", encoding="utf-8") as f:
#             profile = json.load(f)
#         character_name = profile.get("characterName", "This Character")
#         character_bio = profile.get("bio", "No bio available.")
#     except Exception as e:
#         print(f"Error loading profile for {characterId}: {e}")
#         return {"error": f"Could not load character profile: {e}"}

#     # 2. Load character-specific FAISS index and texts
#     char_faiss_index = None
#     char_stored_texts = []
#     try:
#         if (char_path / "index.faiss").exists() and (char_path / "texts.json").exists():
#             char_faiss_index = faiss.read_index(str(char_path / "index.faiss"))
#             with open(char_path / "texts.json", "r", encoding="utf-8") as f:
#                 char_stored_texts = json.load(f)
#         else:
#             print(f"FAISS index or texts.json not found for character {characterId}. Proceeding without RAG for initial data.")
#     except Exception as e:
#         print(f"Error loading FAISS index/texts for {characterId}: {e}. Proceeding without RAG for initial data.")
#         char_faiss_index = None # Ensure it's None if loading failed
#         char_stored_texts = []

#     # Determine user message (text or from audio)
#     actual_user_message = ""
#     transcript_for_response = None

#     if user_audio_file:
#         audio_bytes = await user_audio_file.read()
#         actual_user_message = convert_audio_to_text(audio_bytes)
#         transcript_for_response = actual_user_message # For chat response
#     elif user_message_text:
#         actual_user_message = user_message_text
#     else:
#         return {"error": "You must provide either a text message or an audio file."}

#     if not actual_user_message.strip(): # Check if message is empty after STT or if input was empty
#          return {"error": "User message is empty."}


#     # 3. RAG search using character-specific index
#     relevant_context = []
#     if char_faiss_index is not None and char_stored_texts and actual_user_message:
#         try:
#             query_embedding = embedding_model.encode([actual_user_message])
#             if query_embedding.ndim == 1:
#                 query_embedding = np.asarray([query_embedding])

#             # Ensure k is not greater than the number of items in the index
#             k_search = min(3, char_faiss_index.ntotal)
#             if k_search > 0:
#                 _, indices = char_faiss_index.search(query_embedding, k=k_search)
#                 relevant_context = [char_stored_texts[i] for i in indices[0] if 0 <= i < len(char_stored_texts)]
#         except Exception as e:
#             print(f"Error during FAISS search for {characterId}: {e}")
#             relevant_context = [] # fallback to empty context


#     # 4. Construct Prompts for Llama3
#     system_prompt_content = f"""You are '{character_name}'. Your personality is defined by your biography and answers to a questionnaire.
# Your biography: "{character_bio}".
# You MUST strictly embody the persona of '{character_name}' in all your responses.
# Use the 'Relevant Context' provided below from your knowledge base to answer user questions.
# If the context is empty or doesn't directly answer the question, rely on your established persona ('{character_name}') and biography.
# If you truly don't know the answer and it's not inferable from your persona, you can state that you don't recall or don't have that information.
# Maintain a natural, conversational tone appropriate for '{character_name}'.
# The user is addressing you, '{character_name}'.
# """
#     if is_call: # Slight variation for calls if needed, or can be merged more
#         system_prompt_content += "\nYou are currently in a phone conversation."


#     chat_history_list = []
#     if chat_history_str:
#         try:
#             # Expecting a JSON string array: '["user: Hi", "bot: Hello"]'
#             parsed_history = json.loads(chat_history_str)
#             if isinstance(parsed_history, list):
#                 chat_history_list = [str(item) for item in parsed_history] # Ensure all items are strings
#             else: # Fallback for plain string history
#                 chat_history_list = [chat_history_str]
#         except json.JSONDecodeError:
#             print(f"Warning: chat_history_str was not valid JSON. Treating as a single string entry. Content: {chat_history_str[:100]}")
#             chat_history_list = [chat_history_str] # Fallback if not JSON

#     formatted_chat_history = "\n".join(chat_history_list)

#     user_prompt_content = f"""Relevant Context for '{character_name}' (prioritize this):
# {chr(10).join(relevant_context) if relevant_context else "No specific context found for this query."}

# Recent Conversation History (User and '{character_name}'):
# {formatted_chat_history if formatted_chat_history else "No recent conversation history provided."}

# User's current message to '{character_name}': {actual_user_message}
# Now, as '{character_name}', provide your answer:"""

#     # 5. Call Llama3 (Groq API)
#     headers = {
#         "Authorization": f"Bearer {GROQ_API_KEY}",
#         "Content-Type": "application/json"
#     }
#     payload = {
#         "model": "llama-3.3-70b-versatile", # Or your preferred Groq model, using 3.1 as per common availability
#         "messages": [
#             {"role": "system", "content": system_prompt_content},
#             {"role": "user", "content": user_prompt_content}
#         ],
#         # "temperature": 0.7,
#         # "max_tokens": 300
#     }

#     try:
#         # response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=payload, timeout=30)
#         response = requests.post("https://api.groq.com/openai/v1/chat/completions", headers=headers, json=payload)

#         response.raise_for_status() # Raise an exception for HTTP errors
#         reply = response.json()["choices"][0]["message"]["content"]
#     except requests.exceptions.Timeout:
#         reply = f"Sorry, I couldn't get a response from the AI in time. Please try again. ({character_name})"
#     except requests.exceptions.RequestException as e:
#         print(f"Groq API request error: {e}")
#         reply_error_message = "I seem to be having trouble connecting to my thoughts right now."
#         if response:
#             print(f"Groq API response status: {response.status_code}")
#             print(f"Groq API response text: {response.text}")
#             try: # Try to parse error from Groq if available
#                 error_details = response.json().get("error", {}).get("message")
#                 if error_details:
#                     reply_error_message += f" (Details: {error_details})"
#             except json.JSONDecodeError:
#                 pass # No JSON in error response
#         reply = f"{reply_error_message} ({character_name})"
#     except (KeyError, IndexError) as e:
#         print(f"Error parsing Groq API response: {e}")
#         print("Groq response content:", response.text if 'response' in locals() and response else "No response object")
#         reply = f"I received an unusual response, and I'm not sure how to interpret it. ({character_name})"


#     # Long-term memory (storing interactions in FAISS) is temporarily disabled as per suggestion.
#     # If you want to re-enable, uncomment the following and adapt it to save to char_path/index.faiss and char_path/texts.json
#     # And ensure you handle the FAISS index and texts update correctly (read, add, write).
#     """
#     # messages_to_store = [f"user: {actual_user_message}", f"bot: {reply}"]
#     # if char_faiss_index is not None and char_stored_texts is not None: # Check if they exist
#     #     try:
#     #         # Important: This part needs careful implementation if re-enabled
#     #         # You'd need to load embeddings, add new ones, update texts.json, and rewrite the index.
#     #         # This can be complex and might require re-building the index or careful appends if supported.
#     #         # For simplicity, this example assumes you might just append to texts and rebuild.
#     #         # THIS IS A SIMPLIFIED PLACEHOLDER - DO NOT USE AS IS FOR PRODUCTION MEMORY
#     #         new_embeddings = embedding_model.encode(messages_to_store)
#     #         if new_embeddings.ndim == 1: new_embeddings = np.asarray([new_embeddings])
#     #
#     #         char_faiss_index.add(new_embeddings) # Add to existing index
#     #         char_stored_texts.extend(messages_to_store)
#     #
#     #         # Save updated index and texts
#     #         faiss.write_index(char_faiss_index, str(char_path / "index.faiss"))
#     #         with open(char_path / "texts.json", "w", encoding="utf-8") as f:
#     #             json.dump(char_stored_texts, f, ensure_ascii=False, indent=4)
#     #         print(f"Interaction stored for character {characterId}")
#     #
#     #     except Exception as e:
#     #         print(f"Error storing interaction to FAISS for {characterId}: {e}")
#     # else:
#     #     print(f"FAISS index not available for {characterId}, cannot store interaction.")
#     """

#     if is_call:
#         audio_response_path = convert_text_to_audio(reply)
#         if audio_response_path:
#             return FileResponse(audio_response_path, media_type="audio/wav", filename="response.wav")
#         else:
#             return {"error": "Failed to convert bot reply to audio."}
#     else:
#         return {"reply": reply, "transcript": transcript_for_response}

# في ملف main.py

async def process_chat_or_call(
    characterId: str,
    user_message_text: str | None = None,
    user_audio_file: UploadFile | None = None,
    chat_history_str: str | None = None,
    is_call: bool = False
):
    print("\n--- [START] process_chat_or_call ---")
    print(f"Character ID: {characterId}")

    char_path = BASE_CHARACTER_DATA_PATH / characterId
    if not char_path.exists() or not (char_path / "profile.json").exists():
        print("❌ ERROR: Character profile not found.")
        return {"error": "Character not found or profile is missing."}

    # 1. تحميل بيانات الشخصية وملفات RAG
    try:
        with open(char_path / "profile.json", "r", encoding="utf-8") as f:
            profile = json.load(f)
        character_name = profile.get("characterName", "This Character")
        character_bio = profile.get("bio", "No bio available.")

        char_faiss_index = None
        char_stored_texts = []
        if (char_path / "index.faiss").exists():
            char_faiss_index = faiss.read_index(str(char_path / "index.faiss"))
            with open(char_path / "texts.json", "r", encoding="utf-8") as f:
                char_stored_texts = json.load(f)
        print("✅ Character profile and RAG data loaded successfully.")
    except Exception as e:
        print(f"❌ ERROR loading profile/RAG data: {e}")
        return {"error": f"Could not load character data: {e}"}

    # 2. تحديد رسالة المستخدم
    actual_user_message = ""
    if user_message_text:
        actual_user_message = user_message_text
    elif user_audio_file:
        audio_bytes = await user_audio_file.read()
        actual_user_message = convert_audio_to_text(audio_bytes)
    else:
        return {"error": "No user message provided."}

    print(f"User Message: '{actual_user_message}'")

    # 3. البحث في RAG
    relevant_context = []
    if char_faiss_index and char_stored_texts:
        try:
            query_embedding = embedding_model.encode([actual_user_message])
            k_search = min(3, char_faiss_index.ntotal)
            if k_search > 0:
                _, indices = char_faiss_index.search(query_embedding, k=k_search)
                relevant_context = [char_stored_texts[i] for i in indices[0]]
            print(f"🔍 RAG Search found {len(relevant_context)} context snippets.")
        except Exception as e:
            print(f"⚠️ WARNING: RAG search failed: {e}")

    # 4. بناء التعليمات (Prompts) للـ LLM
    system_prompt_content = f"You are '{character_name}'. Your personality is defined by your biography: \"{character_bio}\". Use the provided context to answer. Be natural and in character."

    # ... (باقي بناء البرومبت كما هو في الكود الأصلي) ...
    formatted_chat_history = "\n".join(json.loads(chat_history_str)) if chat_history_str else ""
    user_prompt_content = f"""Relevant Context: {relevant_context}\n\nChat History:\n{formatted_chat_history}\n\nUser: {actual_user_message}\n{character_name}: """

    print("\n--- [PROMPT] System Prompt ---")
    print(system_prompt_content)
    print("\n--- [PROMPT] User Prompt ---")
    print(user_prompt_content)

    # 5. استدعاء Groq API (باستخدام httpx الصحيح)
    reply = "" # تعريف مبدئي
    try:
        print("\n⏳ Calling Groq API...")
        # --- هذا هو الإصلاح الجوهري ---
        reply = await call_groq_api(system_prompt_content, user_prompt_content)
        # -----------------------------
        print(f"✅ Groq API replied with: '{reply}'")

    except Exception as e:
        print(f"❌ ERROR calling Groq API: {e}")
        # في حالة الخطأ، نرجع رسالة خطأ واضحة بدلاً من الرد الفارغ
        reply = "حدث خطأ أثناء محاولة التفكير، الرجاء المحاولة مرة أخرى."

    print("--- [END] process_chat_or_call ---\n")
    return {"reply": reply, "transcript": actual_user_message if user_audio_file else None}


@app.post("/chat")
async def chat_endpoint(
    characterId: str = Form(...),
    message: str = Form(None),
    file: UploadFile = File(None),
    chat_history_str: str = Form(None) # JSON string of chat history
):
    return await process_chat_or_call(
        characterId=characterId,
        user_message_text=message,
        user_audio_file=file,
        chat_history_str=chat_history_str,
        is_call=False
    )

@app.post("/call")
async def call_endpoint(
    characterId: str = Form(...),
    file: UploadFile = File(...) # Audio from user is mandatory for /call
):
    # For /call, chat_history_str is not explicitly part of the request in the suggestion.
    # If needed, it could be added similar to /chat.
    return await process_chat_or_call(
        characterId=characterId,
        user_audio_file=file,
        is_call=True
    )

# --- Server Startup (example) ---
async def main():
    config = uvicorn.Config(app="main:app", host="0.0.0.0", port=8000, log_level="info", reload=True)
    # Note: If you save this file as `main.py`, then `app="main:app"` is correct.
    # `reload=True` is good for development.
    server = uvicorn.Server(config)
    await server.serve()

if __name__ == "__main__":
    # This allows running with `python your_file_name.py`
    # The `nest_asyncio.apply()` at the top helps if uvicorn is run in an already running loop (e.g. Jupyter)
    # For typical CLI execution, uvicorn manages its own loop.
    # You can also run directly with: uvicorn main:app --host 0.0.0.0 --port 8000 --reload
    print("Starting server programmatically...")
    config = uvicorn.Config(app=app, host="0.0.0.0", port=8000, log_level="info") # Simpler config for direct run
    server = uvicorn.Server(config)
    # server.run() # This is synchronous for Server instance
    asyncio.run(server.serve()) # If you want to run the async serve()

INFO:     Started server process [1670]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:8000 (Press CTRL+C to quit)


ngrok public URL: https://3b3f-34-125-53-190.ngrok-free.app
Starting server programmatically...
Processing 673 characters for AI analysis...
Error creating FAISS index for Nasr: name 'embedding_model' is not defined
INFO:     45.244.87.157:0 - "POST /preprocessing_whats_data HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [1670]
